<a href="https://colab.research.google.com/github/Indranil-R/Silver-Badge-Assignments/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Implement a question answering system with RAG, word embedding, vector database, langchain, llm and any other tools

In [1]:
!pip install -qU -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
from os import getenv
from langchain_google_genai import GoogleGenerativeAI
from google import genai

ModuleNotFoundError: No module named 'langchain_google_genai'

In [ ]:
from google.colab import userdata

In [ ]:
llm = GoogleGenerativeAI(model="models/gemini-2.0-flash",google_api_key=userdata.get('GOOGLE_API_KEY'))
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
template = """Question: {question}

Answer: What is the answer?."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "How much is 2+2?"

# print(chain.invoke({"question": question}))

In [ ]:
print(chain.invoke({"question": "now tell me 2+2"}))

In [ ]:
import getpass
import os

from langchain.chat_models import init_chat_model

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

model.invoke("Please do not tel me about yourself!")

## Creating Embedding from Google API and storing into vector DB and then using it to retreive data

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

### Loading the data from Webpage

In [ ]:
# I am fetching the data from SAP page due to API limitations, keeping the data small
loader = WebBaseLoader(
    web_paths=("https://www.sap.com/india/about/company/faq.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Grid__col-12--cMZPy Grid__col-sm-12--JrL2M Grid__col-md-12--tG5nF Grid__col-lg-12--wWsJK Grid__col-xl-12--v6LlR ")
        )
    ),
)
docs = loader.load()

In [ ]:
print(docs)

In [ ]:
!pip install -q langchain_chroma

In [ ]:
# from langchain.vectorstores.chroma import Chroma

### Creating the embedding

In [ ]:
# Google EMbedding model is
# models/text-embedding-004

from google import genai

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

result = client.models.embed_content(
        model="text-embedding-004",
        contents="docs")


### Adding the embedding to Chroma VectorDB

In [ ]:

# check python ver
import sys
print(sys.version)


In [ ]:
!pip install chromadb

In [ ]:
vector_db = Chroma.from_documents(documents=docs, embedding=result)

In [ ]:
# Switching python ver to 3.9 as chroma is not supported on 3.11
# !sudo apt-get install python3.9

In [ ]:
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1

### Retreive the data

In [ ]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

retriever = MultiQueryRetriever(
    vector_db.as_retriever(search_type="mmr"),
    llm=model,
    prompt=QUERY_PROMPT,

)
